In [9]:
import cv2
import numpy as np
import geocoder
from geopy.distance import geodesic
from tensorflow.keras.models import load_model
import os
import csv
from datetime import datetime
from collections import defaultdict

In [10]:
# === Constants ===
KNOWN_WIDTH_CM = 150
FOCAL_LENGTH_PIXELS = 615
CONFIDENCE_THRESHOLD = 91
MIN_AREA_THRESHOLD = 500
ASPECT_RATIO_MIN = 0.5
ASPECT_RATIO_MAX = 2.0
BRIGHTNESS_THRESHOLD = 240
LOCK_THRESHOLD = 3
IOU_THRESHOLD = 0.5

In [11]:
model = load_model('drone_classifier_no_mapping.h5')
class_names = ['Chinese Drone', 'Not a Drone', 'Turkish Drone']

os.makedirs("detections", exist_ok=True)
csv_file_path = "drone_detections.csv"
if not os.path.exists(csv_file_path):
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Timestamp", "Label", "Confidence", "Distance_m", "Drone_Lat", "Drone_Lon", "Image_Path"])

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def get_current_location():
    g = geocoder.ip('me')  # Offline GPS can replace this
    if g.ok:
        return float(g.latlng[0]), float(g.latlng[1])
    return None, None

def estimate_drone_location(user_lat, user_lon, distance_m):
    origin = (user_lat, user_lon)
    drone_pos = geodesic(kilometers=distance_m / 1000).destination(origin, 0)
    return drone_pos.latitude, drone_pos.longitude

def generate_google_maps_url(user_lat, user_lon, drone_lat, drone_lon):
    return f"https://www.google.com/maps/dir/{user_lat},{user_lon}/{drone_lat},{drone_lon}"

def predict_from_frame(region):
    resized = cv2.resize(region, (224, 224))
    img_arr = resized.astype("float32") / 255.0
    img_arr = np.expand_dims(img_arr, axis=0)
    preds = model.predict(img_arr, verbose=0)
    class_idx = np.argmax(preds[0])
    label = class_names[class_idx]
    confidence = np.max(preds[0]) * 100
    return label, confidence

def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1 + w1, x2 + w2)
    yi2 = min(y1 + h1, y2 + h2)
    inter_width = max(0, xi2 - xi1)
    inter_height = max(0, yi2 - yi1)
    inter_area = inter_width * inter_height
    box1_area = w1 * h1
    box2_area = w2 * h2
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area != 0 else 0

In [12]:
cap0 = cv2.VideoCapture(0)
cap1 = cv2.VideoCapture("http://127.0.0.1:9999/video")

if not cap0.isOpened() or not cap1.isOpened():
    print("Cannot open both cameras.")
    exit()

user_lat, user_lon = get_current_location()
if user_lat is None:
    print("Unable to get current location.")
    cap0.release()
    cap1.release()
    exit()

print("Press 'q' to quit.")
tracking_buffer = defaultdict(int)

def process_frame(frame, cam_id, processed_boxes):
    height, width, _ = frame.shape
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    if len(faces) > 0:
        return frame  # Skip human detection

    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blurred, 200, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        area = cv2.contourArea(contour)
        if area < MIN_AREA_THRESHOLD:
            continue

        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = w / h if h != 0 else 0
        hull = cv2.convexHull(contour)
        hull_area = cv2.contourArea(hull)
        solidity = area / hull_area if hull_area > 0 else 0
        roi_gray = gray[y:y+h, x:x+w]
        avg_brightness = np.mean(roi_gray)

        if not (ASPECT_RATIO_MIN <= aspect_ratio <= ASPECT_RATIO_MAX) or solidity < 0.8:
            continue
        if avg_brightness > BRIGHTNESS_THRESHOLD:
            continue
        if any(calculate_iou((x, y, w, h), box) > IOU_THRESHOLD for box in processed_boxes):
            continue

        roi = frame[y:y+h, x:x+w]
        label, confidence = predict_from_frame(roi)

        if confidence >= CONFIDENCE_THRESHOLD:
            tracking_buffer[label] += 1
            if tracking_buffer[label] >= LOCK_THRESHOLD:
                processed_boxes.append((x, y, w, h))
                distance_cm = (KNOWN_WIDTH_CM * FOCAL_LENGTH_PIXELS) / max(w, 1)
                distance_m = distance_cm / 100.0
                drone_lat, drone_lon = estimate_drone_location(user_lat, user_lon, distance_m)

                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, f"{label} ({confidence:.1f}%) | {distance_m:.1f}m", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

                timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
                image_path = f"detections/cam{cam_id}_frame_{timestamp}.jpg"
                cv2.imwrite(image_path, frame)

                with open(csv_file_path, mode='a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([timestamp, label, f"{confidence:.2f}", f"{distance_m:.2f}",
                                     f"{drone_lat:.5f}", f"{drone_lon:.5f}", image_path])

    return frame

# === Main Loop ===
while True:
    ret0, frame0 = cap0.read()
    ret1, frame1 = cap1.read()
    if not ret0 or not ret1:
        print("Frame capture failed.")
        break

    processed_boxes = []
    frame0 = process_frame(frame0, 0, processed_boxes)
    frame1 = process_frame(frame1, 1, processed_boxes)

    # Resize frames and show side-by-side
    frame0 = cv2.resize(frame0, (640, 480))
    frame1 = cv2.resize(frame1, (640, 480))
    combined = np.hstack((frame0, frame1))
    cv2.imshow("Drone Detection - Laptop & Phone Camera", combined)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap0.release()
cap1.release()
cv2.destroyAllWindows()

Press 'q' to quit.
